In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive #editar esta ruta

In [ ]:
!pip install -q --upgrade transformers scikit-learn pandas

In [ ]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
import matplotlib.pyplot as plt
import joblib

from data.dataset import PeptideDataset
from model.network import BertRegressor

In [ ]:
df = pd.read_csv("data/data_rfu.csv")

scaler = StandardScaler()
df['label'] = scaler.fit_transform(df[['label']])
joblib.dump(scaler, "./saved_model/scaler.pkl")

# Dividir: 81% train / 9% val / 10% test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(
    df['sequence'], df['label'], test_size=0.10, random_state=42
)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_val_texts, train_val_labels, test_size=0.10, random_state=42
)

# Configuracion del tokenizador
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
train_dataset = PeptideDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = PeptideDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)
test_dataset = PeptideDataset(test_texts.tolist(), test_labels.tolist(), tokenizer)

# Configuraracion de DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertRegressor().to(device)
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
loss_fn = nn.MSELoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

In [ ]:
def train(model, train_loader, val_loader, optimizer, loss_fn, device, epochs, patience=5):
    scaler = GradScaler()
    best_val_rmse = float('inf')
    patience_counter = 0
    best_model_path = "./saved_model/best_model.pth"

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Época {epoch+1}", leave=False)

        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            with autocast():
                outputs = model(input_ids, attention_mask)
                loss = loss_fn(outputs, labels)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            progress_bar.set_postfix({'pérdida_lote': loss.item()})

        avg_train_loss = total_loss / len(train_loader)

        #evaluar en validación
        val_rmse, val_mae, val_r2, _, _ = evaluate(model, val_loader, device)
        print(f"📘 Época {epoch+1} | Pérdida de entrenamiento: {avg_train_loss:.4f} | Val RMSE: {val_rmse:.4f} | Val MAE: {val_mae:.4f} | Val R²: {val_r2:.4f}")

        #actualizar programador
        scheduler.step(val_rmse)

        #Detención temprana para evitar sobre ajuste
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Detención temprana en la época {epoch+1}")
                break

    #Cargar el mejor modelo
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    return avg_train_loss

def evaluate(model, test_loader, device):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluando", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(labels.cpu().numpy())

    predictions = np.array(predictions)
    actuals = np.array(actuals)

    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)

    # Guardar resultados
    df_results = pd.DataFrame({
        "real": actuals,
        "predicho": predictions
    })
    df_results.to_csv("./saved_model/test_results.csv", index=False)

    # Graficar predicciones
    plt.figure(figsize=(8, 6))
    plt.scatter(actuals, predictions, alpha=0.5)
    plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'r--', lw=2)
    plt.xlabel("Valores reales (escalados)")
    plt.ylabel("Predicciones (escalados)")
    plt.title("Predicciones vs. Valores reales")
    plt.savefig("./saved_model/predictions_plot.png")
    plt.close()

    return rmse, mae, r2, predictions, actuals

In [ ]:
EPOCHS = 50
train(model, train_loader, val_loader, optimizer, loss_fn, device, EPOCHS)

<ipython-input-26-ec4cb1da0614>:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Época 1:   0%|          | 0/48 [00:00<?, ?it/s]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 1:  10%|█         | 5/48 [00:01<00:13,  3.14it/s, pérdida_lote=0.738]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 1:  29%|██▉       | 14/48 [00:04<00:09,  3.41it/s, pérdida_lote=0.477]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 1:  40%|███▉      | 19/48 [00:05<00:08,  3.34it/s, pérdida_lote=0.606]<ipyth

📘 Época 1 | Pérdida de entrenamiento: 0.8873 | Val RMSE: 0.8526 | Val MAE: 0.7141 | Val R²: 0.2077


Época 2:   0%|          | 0/48 [00:00<?, ?it/s]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 2:   4%|▍         | 2/48 [00:00<00:13,  3.31it/s, pérdida_lote=0.825]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 2:   6%|▋         | 3/48 [00:00<00:13,  3.45it/s, pérdida_lote=1.44]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 2:  12%|█▎        | 6/48 [00:01<00:12,  3.43it/s, pérdida_lote=0.662]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 2:  25%|██▌

📘 Época 2 | Pérdida de entrenamiento: 0.7019 | Val RMSE: 0.7249 | Val MAE: 0.5588 | Val R²: 0.4273


Época 3:   0%|          | 0/48 [00:00<?, ?it/s]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 3:  10%|█         | 5/48 [00:01<00:13,  3.19it/s, pérdida_lote=0.242]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 3:  17%|█▋        | 8/48 [00:02<00:11,  3.45it/s, pérdida_lote=0.442]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 3:  25%|██▌       | 12/48 [00:03<00:10,  3.53it/s, pérdida_lote=0.461]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 3:  27%|█

...

Época 18:   0%|          | 0/48 [00:00<?, ?it/s]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 18:   6%|▋         | 3/48 [00:00<00:14,  3.10it/s, pérdida_lote=0.0306]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 18:  12%|█▎        | 6/48 [00:01<00:13,  3.22it/s, pérdida_lote=0.0433]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 18:  19%|█▉        | 9/48 [00:02<00:12,  3.23it/s, pérdida_lote=0.112]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 18: 

📘 Época 18 | Pérdida de entrenamiento: 0.0820 | Val RMSE: 0.6009 | Val MAE: 0.4624 | Val R²: 0.6064


Época 19:   0%|          | 0/48 [00:00<?, ?it/s]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 19:  10%|█         | 5/48 [00:01<00:12,  3.31it/s, pérdida_lote=0.126]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 19:  12%|█▎        | 6/48 [00:01<00:12,  3.30it/s, pérdida_lote=0.0491]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 19:  25%|██▌       | 12/48 [00:03<00:10,  3.30it/s, pérdida_lote=0.0552]<ipython-input-26-ec4cb1da0614>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Época 19:

📘 Época 19 | Pérdida de entrenamiento: 0.0845 | Val RMSE: 0.5907 | Val MAE: 0.4540 | Val R²: 0.6197
Detención temprana en la época 19


0.08445967696995164

In [ ]:
rmse, mae, r2, preds, actuals = evaluate(model, test_loader, device)
print("\n📌 Evaluación en conjunto de prueba:")
print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")
print(f"R²  : {r2:.4f}")

#Desescalar predicciones y valores reales para interpretación
scaler = joblib.load("./saved_model/scaler.pkl")
actuals_unscaled = scaler.inverse_transform(actuals.reshape(-1, 1)).flatten()
preds_unscaled = scaler.inverse_transform(preds.reshape(-1, 1)).flatten()

# Guardar resultados desescalados
df_results_unscaled = pd.DataFrame({
    "real": actuals_unscaled,
    "predicho": preds_unscaled
})


📌 Evaluación en conjunto de prueba:
RMSE: 0.8251
MAE : 0.5573
R²  : 0.5808
